In [1]:
%%capture
#from jupyterthemes import jtplot
#jtplot.style()
import numpy as np
from landlab import FieldError
from landlab.utils import get_watershed_mask
import xarray as xr
import pandas as pd
import scipy
import gdal
from scipy import ndimage
from scipy.ndimage.filters import *
import os
import math
from osgeo import osr
from fractions import Fraction
import timeit
import matplotlib.pyplot as plt
%matplotlib inline
import random
# import plotting tools
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib as mpl
from landlab.plot.imshow import imshow_grid 
# import necessary landlab components
from landlab import RasterModelGrid, HexModelGrid
from landlab.components import FlowAccumulator, FlowRouter, LakeMapperBarnes, SinkFillerBarnes
from landlab.components import(FlowDirectorD8, 
                               FlowDirectorDINF, 
                               FlowDirectorMFD, 
                               FlowDirectorSteepest)
from landlab.components import DepressionFinderAndRouter
# import landlab plotting functionality
from landlab.plot.drainage_plot import drainage_plot
from pylab import show, figure

def np_array_to_Geotiff(newfile, original_tiff, np_array, dtype):
    
    cols = np_array.shape[1]
    rows = np_array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = original_tiff.GetGeoTransform() 
    driver = gdal.GetDriverByName('GTiff')
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Float64": 
        GDT_dtype = gdal.GDT_Float64
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")
    
    if np_array.ndim == 2:
        band_num = 1
    else:
        band_num = np_array.shape[2]

    outRaster = driver.Create(newfile, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, resample_resolution, 0, originY, 0, -1 * resample_resolution))
    
    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
    
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(np_array)
        else:
            outband.WriteArray(np_array[:,:,b])

    # setteing srs from input tif file.
    prj=original_tiff.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    outRaster = None
    
    return outRaster

# create a plotting routine to make a 3d plot of our surface. 
def surf_plot(mg, surface='topographic__elevation', 
              title='Surface plot of topography', colormap = cm.gray):
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    # Plot the surface.
    Z = (mg.at_node[surface].reshape(mg.shape))#[y1:y2, x1:x2]
    color = colormap((Z-Z.min())/(Z.max()-Z.min()))
    surf = ax.plot_surface(mg.node_x.reshape(mg.shape),#[y1:y2, x1:x2]
                           mg.node_y.reshape(mg.shape),#[y1:y2, x1:x2]
                           Z,
                           rstride=1, cstride=1,
                           facecolors=color,
                           linewidth=0.,
                           antialiased=False)
    ax.view_init(elev=35, azim=-120)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Elevation')
    plt.title(title)
    plt.show()

In [2]:
input_location = (r'C:\PhD\alluvial_fans\gully_heads\all_dems')
output_location = (r'C:\PhD\alluvial_fans\gully_heads\all_catchments')
os.chdir(input_location)
base_resolution = 0.5
for i in range(1, 11):
    resample_resolution = i
    print(resample_resolution)
    os.chdir(input_location)
    for (dirpath, dirnames, filenames) in os.walk('.'):
        for file in filenames:
            if file.endswith('.tif'):
                os.chdir(input_location)
                print(file)
                name = file[:-8]
                print(name)
                input_geotiff = gdal.Open(name + '_dem.tif')
                x = np.array(input_geotiff.GetRasterBand(1).ReadAsArray())
                resample_factor = base_resolution / resample_resolution
                input_DEM = scipy.ndimage.zoom(x, resample_factor, order = 1)
                flow_acc_surf = np.copy(input_DEM).astype('float64');
                rows = flow_acc_surf.shape[0];
                cols = flow_acc_surf.shape[1];
                mg = RasterModelGrid((rows,cols), 1);
                z1 = mg.add_field('topographic__elevation', flow_acc_surf, at = 'node');

                sfb = SinkFillerBarnes(mg, method = 'Steepest', ignore_overfill = True);
                sfb.run_one_step();
                fa = FlowAccumulator(mg,
                                    surface = 'topographic__elevation',
                                    flow_director = 'FlowDirectorDINF');
                #(flow_acc, q) = fa.accumulate_flow();
                fa.run_one_step()
                fd = FlowDirectorDINF(mg, 'topographic__elevation');
                fd.run_one_step()

                da = np.array(mg.at_node['drainage_area'].round(4));
                frn = mg.at_node['flow__receiver_node'];
                drainage_area = da.reshape(mg.shape).astype('float64');
                grid_nodes = (mg.nodes).reshape(mg.shape[0] * mg.shape[1],);
                flow_rec_surf_rows = cols * rows;

                catchment_area = np.multiply(resample_resolution ** 2, drainage_area)  / 10000.0
                output_name = name + '_DINF_' + str(resample_resolution) + 'm.tif'
                print(output_name)
                os.chdir(output_location)
                np_array_to_Geotiff(output_name, input_geotiff, catchment_area, drainage_area.dtype)


1
afm_1_dem.tif
afm_1


C:\Users\sjwal\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


afm_1_DINF_1m.tif
afm_2_dem.tif
afm_2


C:\Users\sjwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.


afm_2_DINF_1m.tif
afm_3_dem.tif
afm_3
afm_3_DINF_1m.tif
afm_4_dem.tif
afm_4
afm_4_DINF_1m.tif
afm_5_dem.tif
afm_5
afm_5_DINF_1m.tif
afm_6_dem.tif
afm_6
afm_6_DINF_1m.tif
afm_7_dem.tif
afm_7
afm_7_DINF_1m.tif
af_10_dem.tif
af_10
af_10_DINF_1m.tif
af_1_dem.tif
af_1
af_1_DINF_1m.tif
af_2_dem.tif
af_2
af_2_DINF_1m.tif
af_3_dem.tif
af_3
af_3_DINF_1m.tif
af_4_dem.tif
af_4
af_4_DINF_1m.tif
af_5_dem.tif
af_5
af_5_DINF_1m.tif
af_6_dem.tif
af_6
af_6_DINF_1m.tif
af_7_dem.tif
af_7
af_7_DINF_1m.tif
af_8_dem.tif
af_8
af_8_DINF_1m.tif
af_9_dem.tif
af_9
af_9_DINF_1m.tif
2
afm_1_dem.tif
afm_1
afm_1_DINF_2m.tif
afm_2_dem.tif
afm_2
afm_2_DINF_2m.tif
afm_3_dem.tif
afm_3
afm_3_DINF_2m.tif
afm_4_dem.tif
afm_4
afm_4_DINF_2m.tif
afm_5_dem.tif
afm_5
afm_5_DINF_2m.tif
afm_6_dem.tif
afm_6
afm_6_DINF_2m.tif
afm_7_dem.tif
afm_7
afm_7_DINF_2m.tif
af_10_dem.tif
af_10
af_10_DINF_2m.tif
af_1_dem.tif
af_1
af_1_DINF_2m.tif
af_2_dem.tif
af_2
af_2_DINF_2m.tif
af_3_dem.tif
af_3
af_3_DINF_2m.tif
af_4_dem.tif
af_4
af_4_DINF_

In [3]:
print('Done')

Done
